In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [6]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

提供された売上データに基づいて、いくつかの傾向を分析します。以下に示すのは、主な分析対象のポイントです。

### 1. カテゴリごとの売上分析
- **食品、衣服、日用品**の3つのカテゴリがあります。それぞれのカテゴリごとの売上・利益量を計算することで、どのカテゴリが最も利益を生んでいるかを把握できます。
- カテゴリごとの総売上、総数量、総原価を集計し、利益（売上 - 原価）を求めましょう。

### 2. 売上推移の可視化
- 売上日ごとの売上トレンドを把握するために、月ごとの売上推移を可視化することが重要です。これにより、季節性や特定の時期に売上が増減する傾向を把握できます。
- グラフを使って、月ごとの売上額や数量を表示し、特定のイベントやキャンペーンの効果を評価します。

### 3. 商品別のパフォーマンス
- 各商品の売上、数量、利益率を評価することで、どの製品が最も効率的に利益を生んでいるかを把握できます。
- 売上が高いにもかかわらず原価が高い商品（利益率が低い）や、逆に売上は低いが高利益の商品に注目することで、戦略的な改善点を見つけることができます。

### 4. 客単価の分析
- 客単価（売上合計/販売数量）を算出することで、購入傾向を把握します。これにより、単価を上げるための施策（バンニングやクロスセリングなど）を検討できます。

### 5. 在庫管理と需要予測
- 売上データを用いて、需要予測モデルを構築することで、在庫管理や仕入れの最適化が期待できます。特に人気商品やシーズン商品についての需要予測は重要です。

### 6. 増加傾向と減少傾向
- どのカテゴリまたは商品の売上が増加しているのか、または減少しているのかを特定して、改善点を見つけます。たとえば、特定の商品のトレンドが悪化している場合、マーケティング戦略や商品ラインナップの見直しが必要かもしれません。

これらの分析を実施することで、企業の方針やマーケティング戦略の見直しが可能となり、成長を促進するための具体的なインサイトを得ることができるでしょう。データを用いながら具体的な処置を取るには、さらに具体的な数値分析や視覚化ツール（PythonのPandasやMatplotlib、Seabornなど）を活用することをお勧めします。


In [7]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   提供された売上データに基づいて、いくつかの傾向を分析します。以下に示すのは、主な分析対象のポ...
1                                                    
2                                  ### 1. カテゴリごとの売上分析
3   - **食品、衣服、日用品**の3つのカテゴリがあります。それぞれのカテゴリごとの売上・利益...
4        - カテゴリごとの総売上、総数量、総原価を集計し、利益（売上 - 原価）を求めましょう。
5                                                    
6                                     ### 2. 売上推移の可視化
7   - 売上日ごとの売上トレンドを把握するために、月ごとの売上推移を可視化することが重要です。こ...
8   - グラフを使って、月ごとの売上額や数量を表示し、特定のイベントやキャンペーンの効果を評価します。
9                                                    
10                                 ### 3. 商品別のパフォーマンス
11  - 各商品の売上、数量、利益率を評価することで、どの製品が最も効率的に利益を生んでいるかを把...
12  - 売上が高いにもかかわらず原価が高い商品（利益率が低い）や、逆に売上は低いが高利益の商品に...
13                                                   
14                                      ### 4. 客単価の分析
15  - 客単価（売上合計/販売数量）を算出することで、購入傾向を把握します。これにより、単価を上...
16                                                   
17                          

In [8]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)